# [실습] 타이타닉 생존자 예측하기

### 목표
- 타이타닉 데이터를 이용하여 데이터 **변수간의 상관관계**를 확인해 본다.
- 사이킷런 분류 모델을 이용하여 타이타닉 **생존자를 예측하는 모델**을 만들어 본다.
- **모델을 평가**해 본다.

### 머신러닝 모델(분류)
- **로지스틱 회귀** 

### 평가지표
- 정확도(accuracy_score)
- 오차행렬(confusion_matrix)
- 정밀도(precision_score)
- 재현율(recall_score)
- F1점수(f1_score)

## 1. 데이터 준비하기

- 다운로드:  https://www.kaggle.com/competitions/titanic/data 
- train.csv : 모델 학습에 사용되는 데이터 파일
- test.csv : 예측에 사용되는 탑승객들의 데이터 파일

In [ ]:
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
# plt.rc('font', family='AppleGothic')            # for 맥 노트북 사용자
plt.rcParams['font.family'] = 'Malgun Gothic'  # for 윈도우 노트북 사용자
plt.rcParams['axes.unicode_minus'] = False     # 그래프에서 (-)숫자표시
plt.rcParams['figure.figsize'] = (5,3)

DIR = './data/titanic/'
train = pd.read_csv(DIR+'train.csv')
test = pd.read_csv(DIR+'test.csv')
submission = pd.read_csv(DIR+'gender_submission.csv')
train.head()

----------------------

## 2. 데이터 전처리

### 2-1. 기본 정보 확인

#### 데이터의 개수 확인

#### 컬럼정보 

| 필드명 | 설명 | 값 종류 |
|------|------|------|
|PassengerId |승객번호  |	|
|Survived |생존여부  |0=No, 1=Yes|	
|Pclass |티켓 클래스 |1=1st, 2=2nd, 3=3rd|	
|Name |이름  |	|	
|Sex |성별  |male=남, female=여	|	
|Age |나이  |	|	
|SibSp |타이타닉 밖의 형제자매/부부의 수  |	|	
|Parch |타이타닉 밖의 부모/자식의 수 |	|	
|Ticket |티켓 번호  |	|	
|Fare |티켓 가격  |	|	
|Cabin |객실 번호  |	|	
|Embarked |승선항구  |C=Cherbourg, Q=Queenstown, S=Southampton|

### 2-2.상관계수로 연관된 필드 확인
- 상관분석(Correlation Analysus) : 두 변수간에 어떤 선형적인 관계를 가지고 있는지를 분석하는 방법
- 상관계수(Correlation coefficient): 두 개의 변수가 같이 일어나는 강도를 나타내는 수치 
- **-1에서 1**사이의 값을 지닙니다. 
- -1이나 1인 수치는 현실 세계에서 관측되기 힘든 수치입니다. 
- 분야별로 기준을 정하는 것에 따라 달라지겠지만, **보통 0.4이상**이면 두 개의 변수간에 상관성이 있다고 얘기합니다. 

<img src="https://t1.daumcdn.net/cfile/tistory/99DEE1425C6A9F2008" width="400" height="400"/>

#### 컬럼에 대한 상관계수 확인 

In [ ]:
# df.corr()

#### 특정 컬럼과 나머지 컬럼간의 상관계수 확인

In [ ]:
# df.corrwith(특정 컬럼)

#### 범주형 데이터  변환
- 성별(Sex): 0, 1 숫자로 변환

#### 학습데이터 상관계수 시각화하기(seaborn heatmap사용)

In [ ]:
import seaborn as sns
#sns.heatmap(학습데이터 상관계수, annot=True)


### 2-3.결측치 처리

### # train 결측치 처리

#### train.info() : Non-Null Count 를 통해 확인한 내용
- **Age** : 결측치 데이터 삭제한다.
- **Embarked** : 범주형 데이터로 변환한다.(C,Q,S=0,1,2) + 결측치(유사 데이터값으로 채움)

#### Age값 결측치 데이터 삭제

#### Embarked :  범주형 데이터로 변환(C,Q,S=0,1,2) + 결측치(관련데이터 값으로 변환)

### # test 결측치 처리

### 2-4.정규화 & 표준화

- **정규화(Normalization)** : -1 ~ 1 사이의 값으로 변환
- **표준화(Standardization)** : 평균이 0이고 표준편차가 1인 값으로 변환
- 머신러닝 성능 향상을 위한 방법
- 정규화와 표준화는 모두 머신러닝 알고리즘을 훈련시키는데 있어서 사용되는 특성(feature)들이 모두 비슷한 영향력을 행사하도록 값을 변환해주는 기술

In [ ]:
def normalize(x):
    return (x-x.min())/(x.max()-x.min())

def standardscaler(x):
    return (x-x.mean())/x.std()

#### Fare 컬럼 정규화&표준화 : 사이킷런의 정규화표준화 클래스 이용

In [ ]:
from sklearn.preprocessing import StandardScaler

s = StandardScaler()
s.fit(train[['Fare']])
train_fare = s.transform(train[['Fare']])
test_fare = s.transform(test[['Fare']])

# 정규화표준화된 값으로 신규컬럼 추가
train['Fare2'] = train_fare
test['Fare2'] = test_fare

# 데이터 형 변환
train['Fare2'] = train['Fare2'].astype('float64')
test['Fare2'] = test['Fare2'].astype('float64')

#### 최종 종속변수와 독립변수로 지정
- 'Age' 컬럼을 종속변수로 사용하지 않는 이유: 편차가 너무 크고 null값이 너무 많았기 때문.
- **종속변수** : Survived 
- **독립변수** : Pclass, Sex, SibSp, Parch, Fare(Fare2), Embarked     

In [ ]:
data = train[['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare2', 'Embarked','Survived']]
dataT= test[['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare2', 'Embarked']]
data.head()

In [ ]:
dataT.head()

### 2-5. 시각화

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.pairplot(data, hue='Survived')
plt.show()

In [ ]:
data.info()

---------------------

## 3. 모델링

### 3-1. 학습용 데이터 준비

In [ ]:
X = data.iloc[:,:6].values
y = data.iloc[:,6].values
X, y

### 3-2. 학습용, 테스트 데이터 분리하기

### 3-3.모델 훈련 및 테스트, 성능평가

#### # 로직스틱 회귀 : LogisticRegression()
- 분류에 적용 시그이드 함수 사용 
- 평가 지표 : 오차행렬(confusion_matrix) 정확도 사용

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score 
import numpy as np

# 학습모델(분류) : 로지스틱 회귀



# 모델 훈련



# 예측



# 정확도 출력




# 혼돈행렬 시각화



# 평가지표
print(f'정확도 : {accuracy_score(y, y_pred)}')
print(f'정밀도 : {precision_score(y, y_pred)}')
print(f'재현율 : {recall_score(y, y_pred)}')
print(f'F1    : {f1_score(y, y_pred)}')
print(f'ROC_ACU : {roc_auc_score(y, y_pred)}')

#### 테스트 데이터 넣어서 예측하기

-------------------

THE END